# Image processing with tensorflow

This notebook will show how the images in a dataset can be processed using both geometric and colour operations.

It will also present a new way of loading data.

### Refs

TensorFlow addons: https://www.tensorflow.org/addons

command: pip install tensorflow-addons

Loading custom datasets: https://www.tensorflow.org/tutorials/load_data/images

In [3]:
import tensorflow as tf

# this package will be used for the data augmentation operations
import tensorflow_addons as tfa

import numpy as np

import os
import pathlib
import matplotlib.pyplot as plt

### Settings

In [4]:
data_dir = 'dogsVScats'

IMG_SIZE = 128

### Aux function to show images

In [5]:
def show_images(image_batch, label_batch, rows, columns):
  count = rows* columns  
  plt.figure(figsize=(10, 2 * rows))
  for n in range(count):
      ax = plt.subplot(int(rows), columns, n+1)
      plt.imshow((image_batch[n]))
      plt.title(classNames[label_batch[n]==1][0])
      plt.axis('off')

In [6]:
BATCH_SIZE = 8

dataset = tf.keras.preprocessing.image_dataset_from_directory(data_dir, image_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, label_mode='categorical', shuffle=True)

normalize = tf.keras.layers.Rescaling(1.0/255)

dataset = dataset.map(lambda x,y: (normalize(x),y))

classNames = np.array(os.listdir(data_dir))

Found 16 files belonging to 2 classes.


AttributeError: module 'tensorflow.keras.layers' has no attribute 'Rescaling'

In [ ]:
image_batch, label_batch = next(iter(dataset))

show_images(image_batch, label_batch, 1, 8)

In [8]:
def process_image(image, label):
    
    r = tf.random.uniform(shape=(), minval=0, maxval=0.5) - 0.25
    
    image = tfa.image.rotate(image, r)
    
    rx = tf.random.uniform(shape=(), minval=0, maxval=20) - 10
    ry = tf.random.uniform(shape=(), minval=0, maxval=20) - 10
    
    image = tfa.image.translate(image, (rx, ry))
    image = tfa.image.random_hsv_in_yiq(image, 0.4, 0.4, 1.4, 0.4, 1.4)
    
    return image, label

In [ ]:
datasetA = dataset.map(process_image)

image_batch, label_batch = next(iter(dataset))
show_images(image_batch, label_batch, 1, 8)

image_batch, label_batch = next(iter(datasetA))
show_images(image_batch, label_batch, 1, 8)

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(data_dir, image_size=(IMG_SIZE, IMG_SIZE), batch_size=None, label_mode='categorical', shuffle=True)

normalize = tf.keras.layers.Rescaling(1.0/255)

datasetB = dataset.map(lambda x,y: (normalize(x), y))

datasetB = datasetB.map(process_image)

datasetB = datasetB.batch(batch_size=BATCH_SIZE)

In [ ]:
image_batch, label_batch = next(iter(datasetB))
show_images(image_batch, label_batch, 1, 8)

image_batch, label_batch = next(iter(datasetB))
show_images(image_batch, label_batch, 1, 8)

In [ ]:
def process_image_trans(image, label):
    
    rx = tf.random.uniform(shape=(), minval=0, maxval=20) - 10
    ry = tf.random.uniform(shape=(), minval=0, maxval=20) - 10
    
    